### Image statistics

In [1]:
import matlab.engine
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
import utils.utils as uti
import process.file as f
import os

### Plotting pixel level image statistics

In [6]:
def get_stats_FOCO(folder):
    df = pd.DataFrame()
    for fold in os.listdir(folder):
        if not fold[0:3] == '.DS':
            for file in os.listdir(folder+'/'+fold):
                if file[-4:] == '.mat':
                    imfile = sio.loadmat(folder+'/'+fold +'/'+file)

                    data = imfile['data']
                    channels = [0,2,4,1]
                    RGBW = np.squeeze(data[:,:,:, channels])

                    RGBW_flat = RGBW.reshape(-1, RGBW.shape[-1])

                    min = np.amin(RGBW, axis = (0,1,2))
                    max = np.amax(RGBW, axis = (0,1,2))
                    mean = np.mean(RGBW, axis = (0,1,2))
                    std = np.std(RGBW, axis = (0,1,2))

                    corr = np.corrcoef(RGBW_flat, rowvar= False)
                    RG_co = corr[0][1]
                    RB_co = corr[0][2]
                    RW_co = corr[0][3]
                    GB_co = corr[1][2]
                    GW_co = corr[1][3]
                    BW_co = corr[2][3]

                    per_ID, per_correct, per_correct2, correctID, corrID2 = uti.check_accuracy(f.proc_FOCO(folder+'/'+fold))

                    df = df.append({'acc':per_correct,'min':min, 'max': max, 'mean': mean, 'std':std, 'RG coef': RG_co, 'RB coef': RB_co, 'RW coef': RW_co, 'GB coef': GB_co, 'GW coef': GW_co, 'BW coef':BW_co}, ignore_index=True)
    return df

def get_stats_NP(folder):
    df = pd.DataFrame()

    for file in os.listdir(folder):
        if file[-9:] == 'YAaLR.mat' or file[-9:] == 'YAaDV.mat':
            imfile = sio.loadmat('data/NP_paper/all/'+file)

            data = imfile['data']
            channels = imfile['prefs']['RGBW'][0][0]-1
            RGBW = np.squeeze(data[:,:,:, channels])/16

            RGBW_flat = RGBW.reshape(-1, RGBW.shape[-1])

            min = np.amin(RGBW, axis = (0,1,2))
            max = np.amax(RGBW, axis = (0,1,2))
            mean = np.mean(RGBW, axis = (0,1,2))
            std = np.std(RGBW, axis = (0,1,2))

            corr = np.corrcoef(RGBW_flat, rowvar= False)
            RG_co = corr[0][1]
            RB_co = corr[0][2]
            RW_co = corr[0][3]
            GB_co = corr[1][2]
            GW_co = corr[1][3]
            BW_co = corr[2][3]

            df = df.append({'min':min, 'max': max, 'mean': mean, 'std':std, 'RG coef': RG_co, 'RB coef': RB_co, 'RW coef': RW_co, 'GB coef': GB_co, 'GW coef': GW_co, 'BW coef':BW_co}, ignore_index=True)
    
    return df

def get_stats_Chaud(folder):
    df = pd.DataFrame()

    for fold in os.listdir(folder):
        if not fold[0:3] == '.DS':
            for file in os.listdir(folder+'/'+fold):
                if file[-4:] == '.mat':
                    imfile = sio.loadmat(folder+'/'+fold +'/'+file)

                    data = imfile['data']
                    channels = imfile['prefs']['RGBW'][0][0]-1
                    RGBW = np.squeeze(data[:,:,:, channels])/16

                    RGBW_flat = RGBW.reshape(-1, RGBW.shape[-1])

                    min = np.amin(RGBW, axis = (0,1,2))
                    max = np.amax(RGBW, axis = (0,1,2))
                    mean = np.mean(RGBW, axis = (0,1,2))
                    std = np.std(RGBW, axis = (0,1,2))

                    corr = np.corrcoef(RGBW_flat, rowvar= False)
                    RG_co = corr[0][1]
                    RB_co = corr[0][2]
                    RW_co = corr[0][3]
                    GB_co = corr[1][2]
                    GW_co = corr[1][3]
                    BW_co = corr[2][3]

                    per_ID, per_correct, per_correct2, correctID, corrID2 = uti.check_accuracy(f.proc_Chaud(folder+'/'+fold))

                    df = df.append({'name':file, 'min':min, 'max': max, 'mean': mean, 'std':std, 'RG coef': RG_co, 'RB coef': RB_co, 'RW coef': RW_co, 'GB coef': GB_co, 'GW coef': GW_co, 'BW coef':BW_co}, ignore_index=True)
    return df
        


In [7]:
FOCO_df = get_stats_FOCO('data/NP_foco_cropped')
NP_df = get_stats_NP('data/NP_paper/all')
chaud_df = get_stats_Chaud('data/NP_chaudhary')


In [10]:
%matplotlib qt

def plot_corr_coefs(data, labels):
    fig, ax = plt.subplots(2,3)
    ax[0,0].boxplot([df['RG coef'] for df in data], labels = labels)
    ax[0,0].set_title('RG corr coef')
    ax[0,1].boxplot([df['RB coef'] for df in data], labels = labels)
    ax[0,1].set_title('RB corr coef')
    ax[0,2].boxplot([df['RW coef'] for df in data], labels = labels)
    ax[0,2].set_title('RW corr coef')
    ax[1,0].boxplot([df['GB coef'] for df in data], labels = labels)
    ax[1,0].set_title('GB corr coef')
    ax[1,1].boxplot([df['GW coef'] for df in data], labels = labels)
    ax[1,1].set_title('GW corr coef')
    ax[1,2].boxplot([df['BW coef'] for df in data], labels = labels)
    ax[1,2].set_title('BW corr coef')

    plt.show()


plot_corr_coefs([FOCO_df, NP_df, chaud_df], ['FOCO', 'NP', 'Chaudhary'])

In [11]:
%matplotlib qt

def plot_corr_acc(df):
    fig, ax = plt.subplots(2,3)

    ax[0,0].scatter(df['RG coef'], df['acc'])
    ax[0,0].set_title('RG coef vs accuracy')
    ax[0,0].set_ylabel('correlation coefficient')
    ax[0,0].set_xlabel('percent accuracy')
    ax[0,1].scatter(df['RB coef'], df['acc'])
    ax[0,1].set_title('RB coef vs accuracy')
    ax[0,1].set_xlabel('percent accuracy')
    ax[0,2].scatter(df['RW coef'], df['acc'])
    ax[0,2].set_title('RW coef vs accuracy')
    ax[0,2].set_xlabel('percent accuracy')
    ax[1,0].scatter(df['GB coef'], df['acc'])
    ax[1,0].set_title('GB coef vs accuracy')
    ax[1,0].set_xlabel('percent accuracy')
    ax[1,0].set_ylabel('correlation coefficient')
    ax[1,1].scatter(df['GW coef'], df['acc'])
    ax[1,1].set_title('GW coef vs accuracy')
    ax[1,1].set_xlabel('percent accuracy')
    ax[1,2].scatter(df['BW coef'], df['acc'])
    ax[1,2].set_title('BW coef vs accuracy')
    ax[1,2].set_xlabel('percent accuracy')

    plt.tight_layout()
    plt.show()

In [8]:
%matplotlib qt

def plot_image_stats(data, labels):

    fig, ax = plt.subplots(2,4)

    for i, df in enumerate(data):

        ax[0,0].scatter(np.stack(df['mean'])[:,0], np.stack(df['mean'])[:,1], label = labels[i], alpha=0.5)
        ax[1,0].scatter(np.stack(df['mean'])[:,2], np.stack(df['mean'])[:,3], label = labels[i], alpha=0.5)
        ax[0,1].scatter(np.stack(df['min'])[:,0], np.stack(df['min'])[:,1], label = labels[i], alpha=0.5)
        ax[1,1].scatter(np.stack(df['min'])[:,2], np.stack(df['min'])[:,3], label = labels[i], alpha=0.5)
        ax[0,2].scatter(np.stack(df['std'])[:,0], np.stack(df['std'])[:,1], label = labels[i], alpha=0.5)
        ax[1,2].scatter(np.stack(df['std'])[:,2], np.stack(df['std'])[:,3], label = labels[i], alpha=0.5)
        ax[0,3].scatter(np.stack(df['max'])[:,0], np.stack(df['max'])[:,1], label = labels[i], alpha=0.5)
        ax[1,3].scatter(np.stack(df['max'])[:,2], np.stack(df['max'])[:,3], label = labels[i], alpha=0.5)

    ax[0,0].set_aspect('equal')
    ax[0,0].set_xlim(0,150)
    ax[0,0].set_ylim(0,150)
    ax[0,0].set_title('RG means')
    ax[0,0].set_xlabel('R channel means')
    ax[0,0].set_ylabel('G channel means')
    ax[0,0].legend()
    ax[1,0].set_aspect('equal')
    ax[1,0].set_xlim(0,150)
    ax[1,0].set_ylim(0,150)
    ax[1,0].set_title('BW means')
    ax[1,0].set_xlabel('B channel means')
    ax[1,0].set_ylabel('W channel means')
    ax[0,1].set_aspect('equal')
    ax[0,1].set_xlim(0,100)
    ax[0,1].set_ylim(0,100)
    ax[0,1].set_title('RG mins')
    ax[0,1].set_xlabel('R channel min')
    ax[0,1].set_ylabel('G channel min')
    ax[1,1].set_aspect('equal')
    ax[1,1].set_xlim(0,100)
    ax[1,1].set_ylim(0,100)
    ax[1,1].set_title('BW min')
    ax[1,1].set_xlabel('B channel min')
    ax[1,1].set_ylabel('W channel min')
    ax[0,2].set_aspect('equal')
    ax[0,2].set_xlim(0,500)
    ax[0,2].set_ylim(0,500)
    ax[0,2].set_title('RG stds')
    ax[0,2].set_xlabel('R channel stds')
    ax[0,2].set_ylabel('G channel stds')
    ax[1,2].set_aspect('equal')
    ax[1,2].set_xlim(0,500)
    ax[1,2].set_ylim(0,500)
    ax[1,2].set_title('BW stds')
    ax[1,2].set_xlabel('B channel stds')
    ax[1,2].set_ylabel('W channel stds')    
    ax[0,3].set_aspect('equal')
    ax[0,3].set_xlim(0,5000)
    ax[0,3].set_ylim(0,5000)
    ax[0,3].set_title('RG max')
    ax[0,3].set_xlabel('R channel max')
    ax[0,3].set_ylabel('G channel max')
    ax[1,3].set_aspect('equal')
    ax[1,3].set_xlim(0,5000)
    ax[1,3].set_ylim(0,5000)
    ax[1,3].set_title('BW max')
    ax[1,3].set_xlabel('B channel max')
    ax[1,3].set_ylabel('W channel max')

    plt.tight_layout()
    plt.show()

In [9]:
plot_image_stats([FOCO_df, NP_df, chaud_df], ['FOCO', 'NP', 'Chaudhary'])

### Histogram evaluation and equalization

In [14]:
def generate_histograms(image, scale):
    
    image = np.asarray(image)

    im_flat = image.reshape(-1, image.shape[-1])

    fig, axs = plt.subplots(2,2)

    for i, ax1 in enumerate(axs):
        for j, ax in enumerate(ax1):

            hist, bins = np.histogram(im_flat[:,i*2+j], 256, [0, 256*scale] )
            cdf = hist.cumsum()
            cdf_normalized = cdf * hist.max()/cdf.max()
            ax.axvline(np.max(im_flat[:,i*2+j]),linestyle='--') 
            ax.plot(cdf_normalized, color = 'b')
            ax.hist(im_flat[:,i*2+j], bins= np.arange(256)*scale, color= 'red')
            ax.set_xlabel('color channel gray count')
            ax.set_ylabel('pixel count')
            ax.set_xlim([0,256*scale])
            ax.legend(('max value', 'cdf', 'hist'), loc = 'upper right')
    
    axs[0,0].set_title('red histogram')
    axs[0,1].set_title('green histogram')
    axs[1,0].set_title('blue histogram')
    axs[1,1].set_title('white histogram')

    plt.show()


In [56]:
%matplotlib qt
imfile = sio.loadmat('data/NP_FOCO_cropped/2021-12-03-w00-NP1/neuropal_1_MMStack_Pos0.ome.mat')
data = imfile['data']
channels = [0,2,4,1]
RGBW = np.squeeze(data[:,:,:, channels])

generate_histograms(RGBW, 16)

In [22]:
%matplotlib qt

imfile = sio.loadmat('data/NP_paper/all/7_YAaLR.mat')
data = imfile['data']
channels = imfile['prefs']['RGBW'][0][0]-1
RGBW = np.squeeze(data[:,:,:, channels])

generate_histograms(RGBW, 256)

In [15]:
def equalize_hist(RGBW, threshs):
    '''
    thresh defines value above which to perform the histogram equalization
    loop through each pixel in image and transform based on histogram equalization
    '''

    size = RGBW.shape

    RGBW_new = np.zeros(size)

    flat = RGBW.reshape(-1, RGBW.shape[-1])

    for l in range(size[3]):
        channel = flat[:,l]

        thresh = threshs[l]
        
        hist_to_eq = channel[np.where(channel>=thresh)]
        N = len(hist_to_eq)
        num_bins = 4096-thresh
        hist, bins = np.histogram(hist_to_eq, num_bins, [thresh, 4096])
        cdf = hist.cumsum()
        
        for i in range(size[0]):
            for j in range(size[1]):
                for k in range(size[2]):
                        val = RGBW[i,j,k,l]

                        if val >= thresh:
                            val_index = np.where(bins==val)
                            cum_prob = cdf[val_index]/N
                            new_val = np.round(cum_prob*(num_bins-1))+thresh

                            RGBW_new[i,j,k,l] = new_val
                        
                        else:
                            RGBW_new[i,j,k,l] = val

    return RGBW_new

In [16]:
import utils.utils as uti
import process.file as f

hist_df = pd.DataFrame()

for folder in os.listdir('data/NP_foco_cropped'):
    if not folder[0:3] == '.DS':
        for file in os.listdir('data/NP_foco_cropped/'+folder):
            if file[-4:] == '.mat':
                imfile = sio.loadmat('data/NP_foco_cropped/'+folder +'/'+file)

                data = imfile['data']
                channels = [0,2,4,1]
                RGBW = np.squeeze(data[:,:,:, channels])
                RGBW = RGBW.astype('int32')

                RGBW_shift = np.maximum(np.zeros(RGBW.shape), RGBW-80) #moves all values back -80 and then truncates to 0

                RGBW_flat = RGBW.reshape(-1, RGBW.shape[-1])

                RGBW_new = equalize_hist(RGBW_shift, [100, 240, 80, 50])

                min = np.amin(RGBW_new, axis = (0,1,2))
                max = np.amax(RGBW_new, axis = (0,1,2))
                mean = np.mean(RGBW_new, axis = (0,1,2))
                std = np.std(RGBW_new, axis = (0,1,2))

                corr = np.corrcoef(RGBW_flat, rowvar= False)
                RG_co = corr[0][1]
                RB_co = corr[0][2]
                RW_co = corr[0][3]
                GB_co = corr[1][2]
                GW_co = corr[1][3]
                BW_co = corr[2][3]

                per_ID, per_correct, per_correct2, correctID, corrID2 = uti.check_accuracy(f.proc_FOCO('data/NP_foco_cropped/'+folder))

                hist_df = hist_df.append({'folder': folder, 'acc':per_correct,'min':min, 'max': max, 'mean': mean, 'std':std, 'RG coef': RG_co, 'RB coef': RB_co, 'RW coef': RW_co, 'GB coef': GB_co, 'GW coef': GW_co, 'BW coef':BW_co}, ignore_index=True)

                sio.savemat('data/NP_FOCO_hist/'+folder+'/'+'hist_equal_image.mat', {'Hist_RGBW':RGBW_new})


0
1
2
3
0
1
2
3
0
1
2
3
0


In [ ]:
%matplotlib qt
plot_image_stats([FOCO_df, hist_df, NP_df], ['FOCO original', 'FOCO Hist Eq', 'NP paper'])


NameError: name 'hist_df' is not defined